# python 第13周作业
    
    ·计算所有年份冬季（12、1、2月）平均；
    ·对中东太平洋海温进行EOF分解，画出第一模态；
    ·对印度洋海温进行EOF分解，画出第一模态；
    ·计算两个第一模态时间系数与我国气温的协方差、相关系数；
    ·分别用印度洋海温和中东太平洋海温与我国气温进行SVD分析；
    ·利用两个第一模态的时间系数回归中国夏季气温。

In [4]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
from matplotlib.ticker import MultipleLocator
from eofs.standard import Eof

In [2]:
plt.rcParams['font.sans-serif'] = ['SimHei']  # 防止无法显示中文并设置黑体
plt.rcParams['axes.unicode_minus'] = False  # 用来正常显示负号

In [3]:
# 读取文件
ds = xr.open_dataset(r'E:\Work\Python\test9\sst.mnmean.nc')['sst']
ds

<xarray.DataArray 'sst' (time: 2016, lat: 89, lon: 180)>
[32296320 values with dtype=float32]
Coordinates:
  * lat      (lat) float32 88.0 86.0 84.0 82.0 80.0 ... -82.0 -84.0 -86.0 -88.0
  * lon      (lon) float32 0.0 2.0 4.0 6.0 8.0 ... 350.0 352.0 354.0 356.0 358.0
  * time     (time) datetime64[ns] 1854-01-01 1854-02-01 ... 2021-12-01
Attributes:
    long_name:     Monthly Means of Sea Surface Temperature
    units:         degC
    var_desc:      Sea Surface Temperature
    level_desc:    Surface
    statistic:     Mean
    dataset:       NOAA Extended Reconstructed SST V5
    parent_stat:   Individual Values
    actual_range:  [-1.8     42.32636]
    valid_range:   [-1.8 45. ]

## 计算所有年份冬季平均

In [7]:
data = ds.loc[ds.time.dt.month.isin([12,1,2])].loc[:, :, :]
data = data.groupby(data.time.dt.year).mean('time')
data

<xarray.DataArray 'sst' (year: 168, lat: 89, lon: 180)>
array([[[-1.7999998, -1.7999998, -1.7999998, ..., -1.7999998,
         -1.7999998, -1.7999998],
        [-1.7999998, -1.7999998, -1.7999998, ..., -1.7999998,
         -1.7999998, -1.7999998],
        [-1.7999998, -1.7999998, -1.7999998, ..., -1.7999998,
         -1.7999998, -1.7999998],
        ...,
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan]],

       [[-1.7999998, -1.7999998, -1.7999998, ..., -1.7999998,
         -1.7999998, -1.7999998],
        [-1.7999998, -1.7999998, -1.7999998, ..., -1.7999998,
         -1.7999998, -1.7999998],
        [-1.7999998, -1.7999998, -1.7999998, ..., -1.7999998,
         -1.7999998, -1.7999998],
...
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan]],

       [[-1.7999998, -1.7999998, -1.7999998, ..., -1.7999998,
         -1.7999998, -1.7999998],
        [-1.7999998, -1.7999998, -1.7999998, ..., -1.7999998,
         -1.7999998, -1.7999998],
        [-1.7999998, -1.7999998, -1.7999998, ..., -1.7999998,
         -1.7999998, -1.7999998],
        ...,
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan]]], dtype=float32)
Coordinates:
  * lat      (lat) float32 88.0 86.0 84.0 82.0 80.0 ... -82.0 -84.0 -86.0 -88.0
  * lon      (lon) float32 0.0 2.0 4.0 6.0 8.0 ... 350.0 352.0 354.0 356.0 358.0
  * year     (year) int64 1854 1855 1856 1857 1858 ... 2017 2018 2019 2020 2021

## 对中东太平洋海温进行EOF分解，画出第一模态